# Voyages API Example

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
Set your subscription key acquired here: https://apis.signalocean.com/profile

In [ ]:
pip install signal-ocean

In [1]:
signal_ocean_api_key = '' #replace with your subscription key

## Call the Voyages API
The Voyages API retrieves information about vessel voyages.

In [2]:
from signal_ocean import Connection
from signal_ocean.voyages import VoyagesAPI
import pandas as pd
from datetime import date, timedelta

In [3]:
connection = Connection(signal_ocean_api_key)
api = VoyagesAPI(connection)

### Get voyages for vessel

In [4]:
vessel_voyages = api.get_voyages(imo=9292187)

In [5]:
vessel_voyages_df = pd.DataFrame([v.__dict__ for v in vessel_voyages])
vessel_voyages_df.tail(5)

,imo,voyage_number,vessel_type_id,vessel_class_id,vessel_status_id,commercial_operator_id,deleted,events,id,vessel_name,...,cargo_group,cargo_type_source,quantity,laycan_from,laycan_to,fixture_status_id,fixture_status,fixture_date,fixture_is_coa,fixture_is_hold
47,9292187,48,1,84,1,1684,False,"(VoyageEvent(id=None, port_id=3303, voyage_id=...",9292187.48,Seaking,...,Dirty,MarketInfo,260000.0,2019-12-20 00:00:00+00:00,2019-12-22 23:59:00+00:00,1.0,FullyFixed,2019-11-20 10:49:57.117000+00:00,False,False
48,9292187,49,1,84,1,1684,False,"(VoyageEvent(id=None, port_id=3317, voyage_id=...",9292187.49,Seaking,...,Dirty,Estimated,NaN,2020-04-25 03:47:05+00:00,2020-04-26 03:47:05+00:00,1.0,FullyFixed,2020-03-19 03:42:23+00:00,False,False
49,9292187,50,1,84,1,1713,False,"(VoyageEvent(id=None, port_id=6615, voyage_id=...",9292187.50,Seaking,...,Dirty,Estimated,NaN,2020-06-01 03:57:15+00:00,2020-06-02 03:57:15+00:00,1.0,FullyFixed,2020-05-26 17:49:57+00:00,True,True
50,9292187,51,1,84,1,1713,False,"(VoyageEvent(id=None, port_id=3317, voyage_id=...",9292187.51,Seaking,...,Dirty,Estimated,NaN,NaT,NaT,NaN,None,NaT,None,None
51,9292187,52,1,84,1,1713,False,"(VoyageEvent(id=None, port_id=6649, voyage_id=...",9292187.52,Seaking,...,Dirty,MarketInfo,260000.0,2021-02-02 00:00:00+00:00,2021-02-02 00:00:00+00:00,1.0,FullyFixed,2021-01-12 13:45:27.563000+00:00,False,False


In [6]:
vessel_events_df = pd.DataFrame(e.__dict__ for voyage_events in vessel_voyages_df['events'] for e in voyage_events)
vessel_events_df.tail(5)

,id,port_id,voyage_id,event_type,event_horizon,purpose,arrival_date,sailing_date,latitude,longitude,...,area_idlevel0,area_name_level0,area_idlevel1,area_name_level1,area_idlevel2,area_name_level2,area_idlevel3,area_name_level3,low_ais_density,event_details
295,None,13261,None,Stop,Historical,Stop,2021-01-25 11:31:10+00:00,2021-01-30 19:44:08+00:00,-13.0024,9.8397,...,24772,Africa Atlantic Coast,37,West Africa,25018,Africa,25027,Africa,None,"(VoyageEventDetail(id=None, event_id=None, eve..."
296,None,3679,None,PortCall,Historical,Load,2021-02-03 07:55:41+00:00,2021-02-04 23:39:34+00:00,3.1606,6.8317,...,24772,Africa Atlantic Coast,37,West Africa,25018,Africa,25027,Africa,True,"(VoyageEventDetail(id=None, event_id=None, eve..."
297,None,3422,None,Stop,Historical,Stop,2021-02-06 11:35:06+00:00,2021-02-08 10:46:06+00:00,-0.9968,7.5479,...,24772,Africa Atlantic Coast,37,West Africa,25018,Africa,25027,Africa,None,"(VoyageEventDetail(id=None, event_id=None, eve..."
298,None,3679,None,PortCall,Historical,Load,2021-02-09 09:05:48+00:00,2021-02-28 23:06:13.400000+00:00,4.1704,7.2187,...,24772,Africa Atlantic Coast,37,West Africa,25018,Africa,25027,Africa,True,"(VoyageEventDetail(id=None, event_id=None, eve..."
299,None,3532,None,PortCall,Future,Discharge,2021-03-24 23:06:30.159000+00:00,2021-03-27 23:00:00+00:00,22.4842,69.6743,...,24764,Pakistan / West Coast India,25012,India / Pakistan,25023,India / Pakistan,84,East,True,None


In [7]:
vessel_event_details_df = pd.DataFrame(e.__dict__ for event_details in vessel_events_df['event_details'] for e in event_details or [])
vessel_event_details_df.tail(5)

,id,event_id,event_detail_type,arrival_date,sailing_date,start_time_of_operation,end_time_of_operation,geo_asset_id,geo_asset_name,latitude,longitude,other_vessel_imo,other_vessel_name
269,None,None,Stop,2020-12-29 03:57:13+00:00,2020-12-30 11:58:24+00:00,NaT,NaT,4446,Changi Lightering Zone,1.269630,103.960800,NaN,None
270,None,None,Stop,2021-01-25 11:31:10+00:00,2021-01-30 19:44:08+00:00,NaT,NaT,4393,FPSO Dalia,-13.002350,9.839715,NaN,None
271,None,None,Jetty,2021-02-03 07:55:41+00:00,2021-02-04 23:39:34+00:00,2021-02-03 10:57:50+00:00,2021-02-03 12:15:52+00:00,4369,FPSO Akpo,3.160565,6.831695,NaN,None
272,None,None,Stop,2021-02-06 11:35:06+00:00,2021-02-08 10:46:06+00:00,NaT,NaT,4062,Port Gentil Anchorage,-0.996812,7.547887,NaN,None
273,None,None,Jetty,2021-02-09 09:05:48+00:00,2021-02-28 23:06:13.400000+00:00,2021-02-26 15:27:11+00:00,2021-02-26 15:27:11+00:00,5352,Bonny Offshore Terminal,4.170430,7.218700,NaN,None


### Get voyages for vessel class

In [8]:
vlcc_id = 84
date_from = date.today() - timedelta(days=30)
recent_vlcc_voyages = api.get_voyages(vessel_class_id=vlcc_id, date_from=date_from)

In [9]:
recent_vlcc_voyages = pd.DataFrame([v.__dict__ for v in recent_vlcc_voyages])
recent_vlcc_voyages.tail(5)

,imo,voyage_number,vessel_type_id,vessel_class_id,vessel_status_id,commercial_operator_id,deleted,events,id,vessel_name,...,cargo_group,cargo_type_source,quantity,laycan_from,laycan_to,fixture_status_id,fixture_status,fixture_date,fixture_is_coa,fixture_is_hold
188,9878644,2,1,84,1,-1,False,"(VoyageEvent(id=None, port_id=3597, voyage_id=...",9878644.2,Yoho,...,Not set,Estimated,NaN,2021-04-03 00:38:32.772000+00:00,2021-04-04 14:31:58.513000+00:00,5.0,PossFixed,2021-03-13 23:56:30+00:00,False,False
189,9878826,3,1,84,1,1713,False,"(VoyageEvent(id=None, port_id=3801, voyage_id=...",9878826.3,Babylon,...,Dirty,Estimated,NaN,NaT,NaT,NaN,None,NaT,None,None
190,9878826,4,1,84,1,1713,False,"(VoyageEvent(id=None, port_id=3801, voyage_id=...",9878826.4,Babylon,...,Dirty,Estimated,NaN,NaT,NaT,NaN,None,NaT,None,None
191,9878838,5,1,84,1,1713,False,"(VoyageEvent(id=None, port_id=3801, voyage_id=...",9878838.5,Silverstone St,...,Dirty,Estimated,NaN,NaT,NaT,NaN,None,NaT,None,None
192,9878838,6,1,84,1,1713,False,"(VoyageEvent(id=None, port_id=3801, voyage_id=...",9878838.6,Silverstone St,...,Dirty,MarketInfo,260000.0,2021-04-01 00:00:00+00:00,2021-04-01 00:00:00+00:00,0.0,OnSubs,2021-03-19 17:36:47+00:00,True,False


### Get voyages for vessel type

In [10]:
tankers_id = 1
date_from = date.today() - timedelta(days=10)
recent_tanker_voyages = api.get_voyages(vessel_type_id=tankers_id, date_from=date_from)

In [11]:
recent_tanker_voyages_df = pd.DataFrame([v.__dict__ for v in recent_tanker_voyages])
recent_tanker_voyages_df.tail(5)

,imo,voyage_number,vessel_type_id,vessel_class_id,vessel_status_id,commercial_operator_id,deleted,events,id,vessel_name,...,cargo_group,cargo_type_source,quantity,laycan_from,laycan_to,fixture_status_id,fixture_status,fixture_date,fixture_is_coa,fixture_is_hold
1388,9888742,2,1,86,1,578,False,"(VoyageEvent(id=None, port_id=3599, voyage_id=...",9888742.2,Anwaar Benghazi,...,Not set,Estimated,NaN,2021-04-14 05:34:32.195000+00:00,2021-04-16 03:40:31.651000+00:00,0.0,OnSubs,2021-03-22 15:41:39+00:00,False,False
1389,9891438,3,1,88,1,529,False,"(VoyageEvent(id=None, port_id=3193, voyage_id=...",9891438.3,Energy Apollo,...,None,MarketInfo,35000.0,2021-04-16 00:00:00+00:00,2021-04-18 00:00:00+00:00,1.0,FullyFixed,2021-03-02 02:31:25+00:00,False,False
1390,9895915,3,1,88,1,440,False,"(VoyageEvent(id=None, port_id=3794, voyage_id=...",9895915.3,Eco Santa Monica,...,Not set,Estimated,NaN,2021-03-26 02:57:59.248000+00:00,2021-03-29 14:23:31.879000+00:00,5.0,PossFixed,2021-03-22 15:57:49+00:00,False,False
1391,9899985,2,1,88,1,1713,False,"(VoyageEvent(id=None, port_id=3498, voyage_id=...",9899985.2,Nord Vision,...,Not set,Estimated,30000.0,2021-03-20 00:00:00+00:00,2021-03-30 00:00:00+00:00,1.0,FullyFixed,2021-03-03 15:07:35+00:00,False,False
1392,9909950,8,1,87,1,-1,False,"(VoyageEvent(id=None, port_id=7278, voyage_id=...",9909950.8,Zhong You Hua Yuan 18,...,Dirty,Estimated,NaN,NaT,NaT,NaN,None,NaT,None,None


### Get voyages for vessel in flat format

In [12]:
vessel_voyages_flat = api.get_voyages_flat(imo=9292187)

In [13]:
vessel_voyages_df = pd.DataFrame(v.__dict__ for v in vessel_voyages_flat.voyages)
vessel_voyages_df.tail(5)

,imo,voyage_number,vessel_type_id,vessel_class_id,vessel_status_id,commercial_operator_id,deleted,events,id,vessel_name,...,cargo_group,cargo_type_source,quantity,laycan_from,laycan_to,fixture_status_id,fixture_status,fixture_date,fixture_is_coa,fixture_is_hold
47,9292187,48,1,84,1,1684,False,None,9292187.48,Seaking,...,Dirty,MarketInfo,260000.0,2019-12-20 00:00:00+00:00,2019-12-22 23:59:00+00:00,1.0,FullyFixed,2019-11-20 10:49:57.117000+00:00,False,False
48,9292187,49,1,84,1,1684,False,None,9292187.49,Seaking,...,Dirty,Estimated,NaN,2020-04-25 03:47:05+00:00,2020-04-26 03:47:05+00:00,1.0,FullyFixed,2020-03-19 03:42:23+00:00,False,False
49,9292187,50,1,84,1,1713,False,None,9292187.50,Seaking,...,Dirty,Estimated,NaN,2020-06-01 03:57:15+00:00,2020-06-02 03:57:15+00:00,1.0,FullyFixed,2020-05-26 17:49:57+00:00,True,True
50,9292187,51,1,84,1,1713,False,None,9292187.51,Seaking,...,Dirty,Estimated,NaN,NaT,NaT,NaN,None,NaT,None,None
51,9292187,52,1,84,1,1713,False,None,9292187.52,Seaking,...,Dirty,MarketInfo,260000.0,2021-02-02 00:00:00+00:00,2021-02-02 00:00:00+00:00,1.0,FullyFixed,2021-01-12 13:45:27.563000+00:00,False,False


In [14]:
vessel_events_df = pd.DataFrame(v.__dict__ for v in vessel_voyages_flat.events)
vessel_events_df.tail(5)

,id,port_id,voyage_id,event_type,event_horizon,purpose,arrival_date,sailing_date,latitude,longitude,...,area_idlevel0,area_name_level0,area_idlevel1,area_name_level1,area_idlevel2,area_name_level2,area_idlevel3,area_name_level3,low_ais_density,event_details
295,9292187.52.2,13261,9292187.52,Stop,Historical,Stop,2021-01-25 11:31:10+00:00,2021-01-30 19:44:08+00:00,-13.0024,9.8397,...,None,None,None,None,None,None,None,None,None,None
296,9292187.52.3,3679,9292187.52,PortCall,Historical,Load,2021-02-03 07:55:41+00:00,2021-02-04 23:39:34+00:00,3.1606,6.8317,...,None,None,None,None,None,None,None,None,True,None
297,9292187.52.4,3422,9292187.52,Stop,Historical,Stop,2021-02-06 11:35:06+00:00,2021-02-08 10:46:06+00:00,-0.9968,7.5479,...,None,None,None,None,None,None,None,None,None,None
298,9292187.52.5,3679,9292187.52,PortCall,Historical,Load,2021-02-09 09:05:48+00:00,2021-02-28 23:06:13.400000+00:00,4.1704,7.2187,...,None,None,None,None,None,None,None,None,True,None
299,9292187.52.6,3532,9292187.52,PortCall,Future,Discharge,2021-03-24 23:06:30.159000+00:00,2021-03-27 23:00:00+00:00,22.4842,69.6743,...,None,None,None,None,None,None,None,None,True,None


In [15]:
vessel_event_details_df = pd.DataFrame(v.__dict__ for v in vessel_voyages_flat.event_details)
vessel_event_details_df.tail(5)

,id,event_id,event_detail_type,arrival_date,sailing_date,start_time_of_operation,end_time_of_operation,geo_asset_id,geo_asset_name,latitude,longitude,other_vessel_imo,other_vessel_name
269,9292187.52.1.0,9292187.52.1,Stop,2020-12-29 03:57:13+00:00,2020-12-30 11:58:24+00:00,NaT,NaT,4446,Changi Lightering Zone,1.269630,103.960800,NaN,None
270,9292187.52.2.0,9292187.52.2,Stop,2021-01-25 11:31:10+00:00,2021-01-30 19:44:08+00:00,NaT,NaT,4393,FPSO Dalia,-13.002350,9.839715,NaN,None
271,9292187.52.3.0,9292187.52.3,Jetty,2021-02-03 07:55:41+00:00,2021-02-04 23:39:34+00:00,2021-02-03 10:57:50+00:00,2021-02-03 12:15:52+00:00,4369,FPSO Akpo,3.160565,6.831695,NaN,None
272,9292187.52.4.0,9292187.52.4,Stop,2021-02-06 11:35:06+00:00,2021-02-08 10:46:06+00:00,NaT,NaT,4062,Port Gentil Anchorage,-0.996812,7.547887,NaN,None
273,9292187.52.5.0,9292187.52.5,Jetty,2021-02-09 09:05:48+00:00,2021-02-28 23:06:13.400000+00:00,2021-02-26 15:27:11+00:00,2021-02-26 15:27:11+00:00,5352,Bonny Offshore Terminal,4.170430,7.218700,NaN,None


In [16]:
vessel_voyages_geos_df = pd.DataFrame(v.__dict__ for v in vessel_voyages_flat.geos)
vessel_voyages_geos_df.tail(5)

,id,name,port_id,port_name,country_id,country,area_idlevel0,area_name_level0,area_idlevel1,area_name_level1,area_idlevel2,area_name_level2,area_idlevel3,area_name_level3
203,6507,Terminal Island,3857,Long Beach,240,United States,24741,US West Coast,25017,West Coast North America,25026,West Coast North America,85,Pacific America
204,6163,Vopak Terminal Europoort,3689,Rotterdam,173,Netherlands,24758,Continent,25016,UK Continent,25025,Mediterranean / UK Continent,25028,West
205,5744,Euro Tank Terminal B.v.,3689,Rotterdam,173,Netherlands,24758,Continent,25016,UK Continent,25025,Mediterranean / UK Continent,25028,West
206,6361,Aruba Lightering Zone,3214,San Nicolaas,21,Aruba,24746,Caribs,9,Caribs,25019,Atlantic America,25028,West
207,5745,Maatschap Europoort Terminal,3689,Rotterdam,173,Netherlands,24758,Continent,25016,UK Continent,25025,Mediterranean / UK Continent,25028,West


### Get voyages for vessel class in the flat format

In [17]:
vlcc_id = 84
date_from = date.today() - timedelta(days=30)
recent_vlcc_voyages_flat = api.get_voyages_flat(vessel_class_id=vlcc_id, date_from=date_from)

In [18]:
print('Voyages:', len(recent_vlcc_voyages_flat.voyages))
print('Events:', len(recent_vlcc_voyages_flat.events))
print('EventDetails:', len(recent_vlcc_voyages_flat.event_details))
print('Geos:', len(recent_vlcc_voyages_flat.geos))

Voyages: 193
Events: 794
EventDetails: 345
Geos: 191


### Get voyages for vessel type in the flat format

In [19]:
tankers_id = 1
date_from = date.today() - timedelta(days=10)
recent_tankers_voyages_flat = api.get_voyages_flat(vessel_type_id=tankers_id, date_from=date_from)

In [20]:
print('Voyages:', len(recent_tankers_voyages_flat.voyages))
print('Events:', len(recent_tankers_voyages_flat.events))
print('EventDetails:', len(recent_tankers_voyages_flat.event_details))
print('Geos:', len(recent_tankers_voyages_flat.geos))

Voyages: 1393
Events: 4833
EventDetails: 1426
Geos: 2977


### Get voyages for vessel incrementally 

Initial requests retrieves voyages mathcing the query criteria and a query token that can be used in the subsequent request.

In [21]:
vessel_voyages, next_request_token = api.get_incremental_voyages(imo=9416642)
len(vessel_voyages)

67

Provided the token retrieved for the previous request, the subsequent request retrieves only voyages that have been updated since the previous request. 

In [22]:
incremental_voyages, next_request_token = api.get_incremental_voyages(imo=9416642, incremental_token=next_request_token)
len(incremental_voyages)

0

The retrieved information is used to replace any modify voyages from the dataset. Note that returned voyages marked as deleted are only used to filter out voyages. 

In [23]:
updated_voyage_ids = set(v.id for v in incremental_voyages)
vessel_voyages = [v for v in vessel_voyages if v.id not in updated_voyage_ids]
new_vessel_voyages = [v for v in incremental_voyages if not v.deleted]
vessel_voyages = sorted(vessel_voyages + new_vessel_voyages, key= lambda v: v.voyage_number)
len(vessel_voyages)

67

### Get voyages for vessel class incrementally 

Voyages for vessel class are retieved and updated in the same way incrementally.

In [24]:
vessel_class_id = 84
date_from = date.today() - timedelta(days=30)

In [25]:
voyages, next_request_token = api.get_incremental_voyages(vessel_class_id=vessel_class_id, date_from=date_from)
len(voyages)

193

In [26]:
incremental_voyages, next_request_token = api.get_incremental_voyages(vessel_class_id=vessel_class_id, date_from=date_from, incremental_token=next_request_token)
len(incremental_voyages)

0

In [27]:
updated_voyage_ids = set(v.id for v in incremental_voyages)
voyages = [v for v in voyages if v.id not in updated_voyage_ids]
new_voyages = [v for v in incremental_voyages if not v.deleted]
voyages = sorted(voyages + new_voyages, key= lambda v: (v.imo, v.voyage_number))
len(voyages)

193

### Get voyages for vessel class incrementally in the flat format

Voyages may be retrieved and updated incrementally in the flat format. 

In [28]:
vessel_class_id = 84
date_from = date.today() - timedelta(days=30)

In [29]:
voyages_flat, next_request_token = api.get_incremental_voyages_flat(vessel_class_id=vessel_class_id, date_from=date_from)

In [30]:
voyages = voyages_flat.voyages
events = voyages_flat.events
event_details = voyages_flat.event_details
geos = voyages_flat.geos

In [31]:
len(voyages), len(events), len(event_details), len(geos)

(193, 794, 345, 191)

In [32]:
incremental_voyages_flat, next_request_token = api.get_incremental_voyages_flat(vessel_class_id=vessel_class_id, date_from=date_from, incremental_token=next_request_token)

In [33]:
len(incremental_voyages_flat.voyages)

0

In this case the update step is applied to voyages, events, event details and geos datasets.

In [34]:
updated_voyage_ids = set(v.id for v in incremental_voyages_flat.voyages)
voyages = [v for v in voyages if v.id not in updated_voyage_ids]
new_voyages = [v for v in incremental_voyages_flat.voyages if not v.deleted]
voyages = sorted(voyages + new_voyages, key= lambda v: (v.imo, v.voyage_number))
len(voyages)

193

In [35]:
updated_event_ids = set(e.id for e in events if e.voyage_id in updated_voyage_ids)
events = [e for e in events if e.id not in updated_event_ids]
events = sorted(events + list(incremental_voyages_flat.events), key= lambda e: e.id)
len(events)

794

In [36]:
event_details = [e for e in event_details if e.id not in updated_event_ids]
event_details = sorted(event_details + list(incremental_voyages_flat.event_details), key= lambda e: e.id)
len(event_details)

345

In [37]:
updated_geo_ids = set(g.id for g in incremental_voyages_flat.geos)
geos = [g for g in geos if g.id not in updated_geo_ids] + list(incremental_voyages_flat.geos)
len(geos)

191